In [ ]:
#!/usr/bin/env python
# -*-coding:utf-8 -*-
"""
  ████
██    ██   Datature
  ██  ██   Powering Breakthrough AI
    ██
 
@File    :   brain_tumor_demo_nifti_upload.ipynb
@Authors :   Koh Quan Wei Ivan, Hoki Fung
@Version :   1.0
@Contact :   hello@datature.io
@License :   Apache License 2.0
@Desc    :   Demo notebook for uploading NIfTI files to Datature
             Nexus using Datature Python SDK.
"""

# Uploading NIFTI files to Datature Nexus

The steps are as follows:

1. Sign up for a free Datature account at https://www.datature.io
2. Create a new project on Nexus
3. Go to the Integrations page
4. Choose `Generate New Secret` to get your Secret Key
5. Follow this script to use Datature SDK to upload the NIfTI files to Nexus

---
For NIfTI files, each file is uploaded as a separate 3D volume.
    
* If the axis of orientation is provided, the SDK will upload a series of 2D slices corresponding to the specified orientation.\
    Hence, you will only see one asset containing multiple slices on Nexus.   
    

* If the axis of orientation is not provided, the SDK will upload a series of 2D slices for each orientation (x, y, z). \
    Hence, you will see three sets of 2D slices (e.g. axial, coronal, sagittal) for each scan.
---

For more information about Datature's Python SDK, see https://developers.datature.io/docs/python-sdk.

# Install Datature Library

In [ ]:
! pip install --upgrade datature

# Import Libraries

In [ ]:
import os
import time
import datature

# Example: Uploading all NIFTI files in a folder

Put your NIfTI files into one folder.

* The dataset we used in the tutorial for demonstration and educational purposes can be found here: https://decathlon-10.grand-challenge.org/, but feel free to use your own data.


Create an upload session to upload or update assets.

* The UploadSession class is designed with two methods, add and start, to support simple and quick batch upload. 

* The workflow order should be to first add your assets to the session, and once satisfied, use start to begin the upload process.

* For the full documentation of UploadSession, see https://developers.datature.io/docs/asset-sdk-functions#uploadsession.

For bulk asset upload, Nexus allows users to add up to 5000 assets in one single upload session. 

* If you are planning to upload more than 5000 assets, simply split them up into different batches. 

* Codes to do so are provided in Datature's Developers Documentation: https://developers.datature.io/docs/asset-sdk-functions#upload_session.

In [ ]:
# Edit these accordingly
data_path = "/Users/Demo_BrainTumor/data" # PATH_TO_YOUR_DATA_FOLDER
secret_key = "<secret_key>" # YOUR_PROJECT_SECRET_KEY

In [ ]:
# Initialize an upload session
datature.secret_key = secret_key
upload_session = datature.Asset.upload_session()

In [ ]:
# Add files to the upload session
files = os.listdir(data_path)

num_files = 0

for file in files:
    num_files+=1
    # add file and specify orientation
    # if no orientation is specified, 2D slices for each orientation will be uploaded
    upload_session.add(f"{data_path}/{file}", orientation="z")

In [ ]:
# Start the upload session in the background and retrieve the operation link for status checks.
# This line uploads data to an asset group called "main", which is the default. The value can be changed.

op_link = upload_session.start(["main"], background=True)["op_link"] 

In [ ]:
# Check upload status
while datature.Operation.retrieve(
        op_link)["status"]["progress"]["with_status"]["finished"] != num_files:
    
          print("Uploading...")
          time.sleep(1)
        
print(f"Upload Completed! {num_files} files uploaded to Nexus!")

In [ ]:
# See asset statistics of a specific asset group
datature.Asset.group("main")